# LivecellAction: Visualize EBSS120 hours untreated dataset prediction results

### Loading sample data and create a sample trajectory collection

In [ ]:
from pathlib import Path
from livecellx.sample_data import tutorial_three_image_sys

from livecellx.core.io_sc import prep_scs_from_mask_dataset

from livecellx.core.single_cell import (
    SingleCellStatic,
    SingleCellTrajectory,
    SingleCellTrajectoryCollection,
)


# scs_path = r"D:\LiveCellTracker-dev\notebooks\application_results\5days_notreat_XY03_max_age=3_min_hits=1\5days_notreat_XY03_max_age=3_min_hits=1.json"
# scs_path = (
#     Path(
#         r"D:\LiveCellTracker-dev\notebooks\application_results\Gaohan_5days_notreat\pos_scs"
#     )
#     / "XY03_scs.json"
# )
# scs_path = scs_path.as_posix()
# scs = SingleCellStatic.load_single_cells_json(scs_path)

sctc_path = r"D:\LiveCellTracker-dev\datasets\DIC-Nikon-gt\tifs_CFP_A549_VIM_120hr_NoTreat_NA_YL_Ti2e_2023-03-22\GH-XY03_traj\traj_XY03.json"
sctc = SingleCellTrajectoryCollection.load_from_json_file(sctc_path)

# single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
len(sctc)

In [ ]:
for sc in sctc.get_all_scs():
    sc.mask_dataset = None

In [ ]:
sctc.get_all_scs()[0].show_panel(padding=100)

In [ ]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs,
)


dic_dataset = sctc.get_all_scs()[0].img_dataset
mask_dataset = None


traj_collection = sctc
# traj_collection = track_SORT_bbox_from_scs(
#     scs, dic_dataset, mask_dataset=None, max_age=5, min_hits=1
# )
# traj_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=0, min_hits=1)

### Call livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval to create the interface
If key does not work after you click the slice bar at the bottom, please click the canvas (middle) and try again.

In [ ]:
traj_collection.remove_empty_sct()

In [ ]:
len(traj_collection)

In [ ]:
import tqdm 
dic_dataset.max_cache_size = 1500
for time in tqdm.tqdm(dic_dataset.times):
    dic_dataset.get_img_by_time(time)
    

In [ ]:
traj_collection.remove_empty_sct()
len(traj_collection)

In [ ]:
from livecellx.core.sct_operator import (
    create_scs_edit_viewer,
    SctOperator,
    create_scs_edit_viewer_by_interval,
    _get_viewer_sct_operator,
    create_sctc_edit_viewer_by_interval,
)
import livecellx
import importlib

# sct_opeartor = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(traj_collection, img_dataset=dic_dataset, span_interval=3)
# sct_opeartor = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(traj_collection, img_dataset=dic_dataset, span_interval=3, contour_sample_num=100, viewer=sct_operator.viewer)

In [ ]:
import pandas as pd
# timeseg_csv_path = r"D:\LiveCellTracker-dev\notebooks\notebook_results\Nikon_EBSS_120hr_XY03-timesformer-default-divst-v13-st-combined-random-crop-sequential-20231216-203127\all_mitosis_seg_info.csv"
timeseg_csv_path = r"D:\LiveCellTracker-dev\notebooks\notebook_results\Nikon_EBSS_120hr_XY03-timesformer-default-divst-v13-st-combined-random-crop-20231216-154700\all_mitosis_seg_info.csv"
timeseg_df = pd.read_csv(timeseg_csv_path)
timeseg_df.columns

In [ ]:
importlib.reload(livecellx.core.single_cell)
importlib.reload(livecellx.core.sct_operator)
importlib.reload(livecellx.core.sc_seg_operator)
importlib.reload(livecellx.core.napari_visualizer)

all_tids = timeseg_df["tid"].unique()
pred_mitosis_traj_collection = SingleCellTrajectoryCollection()
for tid in all_tids:
    pred_mitosis_traj_collection.add_trajectory(sctc.get_trajectory(tid))
sct_operator = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(
    pred_mitosis_traj_collection, img_dataset=dic_dataset, span_interval=200, contour_sample_num=15
)

In [ ]:
len(pred_mitosis_traj_collection.get_all_scs())

In [ ]:
def is_mitosis_timeseg(tid, time, timeseg_df) -> bool:
    # timeseg_df's columns are tid, start, end
    # test if a (tid, timeframe) is in [start, end]
    tid_df = timeseg_df[timeseg_df["tid"] == tid]
    return ((tid_df["start"] <= time) & (tid_df["end"] >= time)).any()


In [ ]:
new_properties = sct_operator.shape_layer.properties
face_colors = []
for idx in range(len(sct_operator.shape_layer.properties["sc"])):
    track_id = sct_operator.shape_layer.properties["track_id"][idx]
    sc = sct_operator.shape_layer.properties["sc"][idx]
    new_properties["status"][idx] = ""
    if is_mitosis_timeseg(track_id, sc.timeframe, timeseg_df):
        new_properties["status"][idx] = "mitosis"
        face_colors.append((1, 1, 1, 0.5))
    else:
        face_colors.append((0, 0, 0, 0))
    

# Set the new properties to take effect because the properties attribute is immutable[?]
sct_operator.shape_layer.properties = new_properties
sct_operator.shape_layer.face_color = face_colors

In [ ]:
sct_operator.meta["_contour_sample_num"] = 10
sct_operator.meta["_span_interval"] = 300